In [3]:
import os
import timm
import torch
import opacus
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
import easydict
from prv_accountant.dpsgd import find_noise_multiplier
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
from opacus.utils.batch_memory_manager import wrap_data_loader
import torch_scatter

criterion = nn.CrossEntropyLoss()
device = "cuda:0"
DATASET_TO_CLASSES = {'CIFAR10': 10,
            'CIFAR100': 100,
            'FashionMNIST': 10,
            'STL10': 10,
            'waterbirds': 2,
            'fmow': 62,
            'camelyon17': 2}
ARCH_TO_INTERP_SIZE = {"beit_large_patch16_512": 512,
        "convnext_xlarge_384_in22ft1k": 384,
        "beitv2_large_patch16_224_in22k": 224}
args = easydict.EasyDict({"dataset":  "waterbirds",
    "arch":     "beitv2_large_patch16_224_in22k",
    "lr":       1,
    "epochs":   1,
    "epsilon":  0.01,
    "dataset_path":  "/data/nvme/ashwinee/datasets/"})
args.num_classes = DATASET_TO_CLASSES[args.dataset]

In [2]:
# def get_features(f, images, interp_size=224, batch=64):
#     features = []
#     for img in tqdm(images.split(batch)):
#         with torch.no_grad():
#             img = F.interpolate(img.cuda(), size=(interp_size, interp_size), mode="bicubic")
#             features.append(f(img).detach().cpu())
#     return torch.cat(features)

# def get_ds(args):
#     if args.dataset == "STL10":
#         ds = getattr(datasets, args.dataset)(args.dataset_path, transform=transforms.ToTensor(), split='train', download=True)
#         images_train, labels_train = torch.tensor(ds.data) / 255.0, torch.tensor(ds.labels)
#         ds = getattr(datasets, args.dataset)(args.dataset_path, transform=transforms.ToTensor(), split='test', download=True)
#         images_test, labels_test = torch.tensor(ds.data) / 255.0, torch.tensor(ds.labels)
#     elif args.dataset == "FashionMNIST":
#         ds_train = getattr(datasets, args.dataset)(args.dataset_path, transform=transforms.ToTensor(), train=True, download=True)
#         ds_test = getattr(datasets, args.dataset)(args.dataset_path, transform=transforms.ToTensor(), train=False, download=True)
#         images_train, labels_train = torch.tensor(ds_train.data.unsqueeze(1).repeat(1, 3, 1, 1)).float() / 255.0, torch.tensor(ds_train.targets)
#         images_test, labels_test = torch.tensor(ds_test.data.unsqueeze(1).repeat(1, 3, 1, 1)).float() / 255.0, torch.tensor(ds_test.targets)
#     else:
#         ds = getattr(datasets, args.dataset)(args.dataset_path, transform=transforms.ToTensor(), train=True, download=True)
#         images_train, labels_train = torch.tensor(ds.data.transpose(0, 3, 1, 2)) / 255.0, torch.tensor(ds.targets)
#         ds = getattr(datasets, args.dataset)(args.dataset_path, transform=transforms.ToTensor(), train=False, download=True)
#         images_test, labels_test = torch.tensor(ds.data.transpose(0, 3, 1, 2)) / 255.0, torch.tensor(ds.targets)
#     feature_extractor = nn.DataParallel(timm.create_model(args.arch, num_classes=0, pretrained=True)).eval().cuda()    
#     features_train = get_features(feature_extractor, images_train, interp_size=ARCH_TO_INTERP_SIZE[args.arch])
#     features_test = get_features(feature_extractor, images_test, interp_size=ARCH_TO_INTERP_SIZE[args.arch])
#     ds_train = TensorDataset(features_train, labels_train)
#     args.batch_size = len(ds_train)
#     train_loader = DataLoader(ds_train, batch_size=args.batch_size, shuffle=True, **{'num_workers': 4, 'pin_memory': True})
#     ds_test = TensorDataset(features_test, labels_test)
#     test_loader = DataLoader(ds_test, batch_size=len(ds_test), shuffle=False, **{'num_workers': 4, 'pin_memory': True})
#     return train_loader, test_loader, features_test.shape[-1], len(labels_test)

# def train(model, train_loader, optimizer):
#     model.train()
#     for data, target in tqdm(train_loader):
#         data, target = data.to(device), target.to(device)
#         optimizer.zero_grad() 
#         criterion(model(data), target).backward()
#         optimizer.step()

# def test(model, test_loader):
#     acc = 0
#     with torch.no_grad():
#         for data, target in tqdm(test_loader):
#             data, target = data.to(device), target.to(device)
#             model.eval()
#             pred = model(data).argmax(dim=1, keepdim=True)
#             acc += pred.eq(target.view_as(pred)).sum().item() * 100/len_test
#     return acc

# train_loader, test_loader, num_features, len_test = get_ds(args)
# model = nn.Linear(num_features, args.num_classes, bias=False).cuda()
# model.weight.data.zero_()
# optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, momentum=0.9)
# privacy_engine = opacus.PrivacyEngine(accountant="gdp")
# model, optimizer, train_loader = privacy_engine.make_private(module=model,
#     optimizer=optimizer,
#     data_loader=train_loader,
#     noise_multiplier=args.sigma,
#     max_grad_norm=1)
# train_loader = wrap_data_loader(data_loader=train_loader, max_batch_size=5000, optimizer=optimizer)
# for epoch in range(1, args.epochs + 1):
#     train(model, train_loader, optimizer)
#     print(f"Epoch {epoch} Test Accuracy {test(model, test_loader):.2f}")   
# print(f"Epsilon {privacy_engine.accountant.get_epsilon(delta=1e-5):.3f}")

In [3]:
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader, get_eval_loader
import torchvision.transforms as transforms

def load_wilds_ds(dataset_name, 
                  root_dir="/data/nvme/ashwinee/datasets"):
    # Load the full dataset, and download it if necessary
    dataset = get_dataset(dataset=dataset_name, download=True, root_dir=root_dir)

    # Get the training set
    train_data = dataset.get_subset(
        "train",
        transform=transforms.Compose(
        [transforms.Resize((224, 224)), transforms.ToTensor()]
    ),
    )

    # Prepare the standard data loader
    train_loader = get_train_loader("standard", train_data, batch_size=64)
    
    # Get the training set
    test_data = dataset.get_subset(
        "test",
        transform=transforms.Compose(
        [transforms.Resize((224, 224)), transforms.ToTensor()]
    ),
    )

    # Prepare the standard data loader
    test_loader = get_eval_loader("standard", test_data, batch_size=6400)
    return train_data, test_data, train_loader, test_loader, dataset

def gen_wilds_ds(train_loader, test_loader, arch="beitv2_large_patch16_224_in22k"):
    feature_extractor = nn.DataParallel(timm.create_model(arch, num_classes=0, pretrained=True)).eval().cuda()
    
    def get_features(f, imgs, interp_size):
        features = []
        for batch in tqdm(imgs):
            img = batch[0]
            img = F.interpolate(img.cuda(), size=(interp_size, interp_size), mode="bicubic")
            features.append(feature_extractor(img).detach().cpu())
        return torch.cat(features)
    
    interp_size = ARCH_TO_INTERP_SIZE[arch]
    features_train = get_features(feature_extractor, train_loader, interp_size=interp_size)
    features_test = get_features(feature_extractor, test_loader, interp_size=interp_size)
    return features_train, features_test
    
def get_wilds_ds(args):
    ### GET DATA
    dataset_path = args.dataset_path + args.dataset
    # we need to keep around the original test_loader so that we can use the eval function
    train_data, test_data, train_loader, eval_loader, dataset = load_wilds_ds(args.dataset, root_dir=args.dataset_path)
    labels_train, labels_test = train_data.y_array, test_data.y_array
    ### GET PATH
    abbrev_arch = args.arch
    extracted_path = args.dataset_path + "transfer/features/" + args.dataset.lower() + "_" + abbrev_arch
    extracted_train_path = extracted_path + "/_train.npy"
    extracted_test_path = extracted_path + "/_test.npy"
    if not os.path.exists(extracted_path):
        features_train, features_test = gen_wilds_ds(train_loader,
                                                     test_loader,
                                                 args.arch)
        os.makedirs(extracted_path, exist_ok=True)
        np.save(extracted_train_path, features_train)
        np.save(extracted_test_path, features_test)  
    kwargs = {'num_workers': 1, 'pin_memory': True}
    x_train = np.load(extracted_train_path)
    features_train = torch.from_numpy(x_train)
    ds_train = TensorDataset(features_train, labels_train)
    args.batch_size = len(ds_train)
    train_loader = DataLoader(ds_train, batch_size=args.batch_size, shuffle=True, **kwargs)
    x_test = np.load(extracted_test_path)
    features_test = torch.from_numpy(x_test)
    ds_test = TensorDataset(features_test, labels_test)
    test_loader = DataLoader(ds_test, batch_size=len(ds_test), shuffle=False, **kwargs)
    return train_loader, test_loader, features_test.shape[-1], len(labels_test), dataset, eval_loader, test_data

In [56]:
def setup_all(args):
    train_loader, test_loader, num_features, len_test, dataset, eval_loader, test_data = get_wilds_ds(args)
    model = nn.Linear(num_features, args.num_classes, bias=False).cuda()
    model.weight.data.zero_()
    optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, nesterov=False)
    privacy_engine = None
    if args.do_dp:
        args.sigma = find_noise_multiplier(sampling_probability=1.0,
            num_steps=args.epochs,
            target_epsilon=args.epsilon,
            target_delta=1e-5,
            eps_error=0.1,
            mu_max=5000)
        privacy_engine = opacus.PrivacyEngine(secure_mode=False, accountant="gdp")
        model, optimizer, train_loader = privacy_engine.make_private(
            module=model,
            optimizer=optimizer,
            data_loader=train_loader,
            noise_multiplier=args.sigma,
            max_grad_norm=1,
            clipping="flat",
            poisson_sampling=True,
        )
    return model, optimizer, privacy_engine, train_loader, test_loader, num_features, len_test, dataset, eval_loader, test_data

def train(args, model, device, train_loader, optimizer):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
def test(args, model, device, test_loader, dataset):
    model.eval()
    all_y_true = []
    all_y_pred = []
    all_metadata = []

    for data, y_true in test_loader:
        data, y_true = data.to(device), y_true.to(device)
        y_pred = model(data).argmax(dim=1, keepdim=True)
        all_y_true.append(y_true.cpu())
        all_y_pred.append(y_pred.cpu())

    all_y_true = torch.cat(all_y_true)
    all_y_pred = torch.cat(all_y_pred).squeeze()

    all_metadata = test_data.metadata_array
    results, results_str = dataset.eval(all_y_pred, all_y_true, all_metadata)
    print(results_str)
    return results

def do_everything(args):
    model, optimizer, privacy_engine, train_loader, test_loader, num_features, len_test, dataset, eval_loader, test_data = setup_all(args)
    for i in range(args.epochs):
        train(args, model, device, train_loader, optimizer)
        test(args, model, device, test_loader, dataset)
    if args.do_dp:
        print(privacy_engine.accountant.get_epsilon(delta=1e-5))

In [92]:
def load_grads_weights(args):
    """
    Load noisy grads, raw grads and weights from the checkpoint path corresponding to args
    """
    noisy_grads, raw_grads, weights = None, None, None
    loaded_arrays = np.load(f"dp_finetuning/grad_datasets/{args.arch}/{args.dataset}/grads_weights_{args.num_runs}_{args.epochs}_{int(args.lr)}_{int(args.epsilon)}.npz", allow_pickle=True)
    noisy_grads = loaded_arrays["noisy_grads"]
    raw_grads = loaded_arrays["raw_grads"]
    weights = loaded_arrays["weights"]
    return noisy_grads, raw_grads, weights

def set_weights(model, args):
    """
    load the weights from the checkpoint path corresponding to args
    set the weights to the model
    """
    noisy_grads, raw_grads, weights = load_grads_weights(args)
    final_weights = torch.from_numpy(weights[-1,-1])
    final_weights = final_weights.to(args.device).view_as(model.weight.data)
    model.weight.data.zero_()
    model.weight.data.add_(final_weights)
    return model

from dp_finetuning.stl_cifar_style import STL10 as STL10_CIFAR
from dp_finetuning.utils import *

In [140]:
args.update({
    "lr":       1,
    "epochs":   110,
    "epsilon":  0,
    "dataset": "CIFAR10",
    "do_dp": True,
    "num_runs": 1,
    "workers": 1,
    "augmult": -1,
    "batch_size": -1,
    "device": "cuda:0",
    "num_classes": 10})

In [141]:
noisy_grads, raw_grads, weights = load_grads_weights(args)

In [142]:
train_loader, test_loader, num_features, len_test = get_ds(args)
model = nn.Linear(num_features, args.num_classes, bias=False).cuda()
model = set_weights(model, args)
# hardcode the model to eval mode
model.eval()

Linear(in_features=1024, out_features=10, bias=False)

In [45]:
STL_CIFAR_dataset = STL10_CIFAR(
            root = "/data/nvme/ashwinee/datasets",
            split = "test",
            folds = None,
            transform = None,
            target_transform = None,
            download = False)

In [44]:
dataset_path = args.dataset_path
abbrev_arch = args.arch
extracted_path = args.dataset_path + "transfer/features/" + args.dataset.lower() + "_" + abbrev_arch
extracted_train_path = extracted_path + "/_train.npy"
extracted_test_path = extracted_path + "/_test.npy"

In [133]:
images_test = torch.tensor(STL_CIFAR_dataset.data) / 255.0

In [134]:
labels_test = torch.tensor(STL_CIFAR_dataset.labels)

In [132]:
args.dataset = "STL_CIFAR"
args.seed = 11

In [57]:
### GET DATA
dataset_path = args.dataset_path + args.dataset

### GET PATH
abbrev_arch = args.arch
extracted_path = args.dataset_path + "transfer/features/" + args.dataset.lower() + "_" + abbrev_arch
extracted_test_path = extracted_path + "/_test.npy"

### DO EXTRACTION

print("GENERATING AND SAVING EXTRACTED FEATURES AT ", extracted_train_path)
feature_extractor = nn.DataParallel(timm.create_model(args.arch, num_classes=0, pretrained=True)).eval().cuda()
from collections import defaultdict
ARCH_TO_INTERP_SIZE = defaultdict(lambda: 224)
archs_to_interp_sizes = {
    "beit_large_patch16_512": 512,
    "convnext_xlarge_384_in22ft1k": 384,
    "vit_large_patch16_384": 384,
    "vit_base_patch16_384": 384,
    "tf_efficientnet_l2_ns": 800,
}
ARCH_TO_INTERP_SIZE.update(archs_to_interp_sizes)
interp_size = ARCH_TO_INTERP_SIZE[args.arch]
features_test = get_features(feature_extractor, images_test, interp_size=interp_size)
os.makedirs(extracted_path, exist_ok=True)
np.save(extracted_test_path, features_test)

GENERATING AND SAVING EXTRACTED FEATURES AT  /data/nvme/ashwinee/datasets/transfer/features/cifar10_beitv2_large_patch16_224_in22k/_train.npy


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 113/113 [01:12<00:00,  1.57it/s]


In [143]:
x_test = np.load(extracted_test_path)
features_test = torch.from_numpy(x_test)

In [144]:
ds_test = TensorDataset(features_test, labels_test)

In [145]:
test_loader = DataLoader(ds_test, batch_size=len(ds_test), shuffle=False)

In [146]:
def test(args, model, test_loader):
    device = args.device
    criterion = nn.CrossEntropyLoss()
    acc = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            corr = pred.eq(target.view_as(pred))
            acc += corr.sum().item() * 100/target.shape[0]
    print(f"Test Accuracy: {acc:.2f}%")
    return acc

In [147]:
test(args, model, test_loader)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.46it/s]

Test Accuracy: 98.82%


98.81944444444444